In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
import pandas as pd

df = pd.read_csv("/merged_df.csv")

In [9]:
from datasets import Dataset

data_examples = df.apply(lambda row: {
    "conversations": [
        {"from": "human", "value": row["user_input"]},
        {"from": "gpt", "value": row["reference"]}
    ]
}, axis=1).tolist()

dataset = Dataset.from_list(data_examples)

In [10]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 548
})

In [11]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [12]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'What advancements does DeepSeek-R1-Zero bring to reasoning capabilities in language models?'},
 {'from': 'gpt',
  'value': 'DeepSeek-R1-Zero exhibits super performance on reasoning benchmarks, with a pass@1 score on AIME 2024 increasing from 15.6% to 71.0%, and further improving to 86.7% with majority voting, matching the performance of OpenAI-o1-0912. However, it also faces challenges such as poor readability and language mixing.'}]

In [13]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/548 [00:00<?, ? examples/s]

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

In [14]:
dataset[5]["conversations"]

[{'content': 'What advancements does DeepSeek-R1-Zero bring to reasoning capabilities in language models?',
  'role': 'user'},
 {'content': 'DeepSeek-R1-Zero exhibits super performance on reasoning benchmarks, with a pass@1 score on AIME 2024 increasing from 15.6% to 71.0%, and further improving to 86.7% with majority voting, matching the performance of OpenAI-o1-0912. However, it also faces challenges such as poor readability and language mixing.',
  'role': 'assistant'}]

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/548 [00:00<?, ? examples/s]

In [16]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=2):   0%|          | 0/548 [00:00<?, ? examples/s]

In [17]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat advancements does DeepSeek-R1-Zero bring to reasoning capabilities in language models?<|im_end|>\n<|im_start|>assistant\nDeepSeek-R1-Zero exhibits super performance on reasoning benchmarks, with a pass@1 score on AIME 2024 increasing from 15.6% to 71.0%, and further improving to 86.7% with majority voting, matching the performance of OpenAI-o1-0912. However, it also faces challenges such as poor readability and language mixing.<|im_end|>\n'

In [18]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                             \nDeepSeek-R1-Zero exhibits super performance on reasoning benchmarks, with a pass@1 score on AIME 2024 increasing from 15.6% to 71.0%, and further improving to 86.7% with majority voting, matching the performance of OpenAI-o1-0912. However, it also faces challenges such as poor readability and language mixing.<|im_end|>\n'

In [19]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.879 GB of memory reserved.


In [20]:
df['user_input'][530]

'Can you elaborate on the features and significance of FlashMLA in the context of open-source AI development?'

In [21]:
df['reference'][530]

'FlashMLA is an efficient MLA decoding kernel specifically optimized for Hopper GPUs, designed to handle variable-length sequences and has been battle-tested in production. It boasts several notable features, including BF16 support, a paged KV cache with a block size of 64, and impressive performance metrics, achieving 3000 GB/s memory-bound and 580 TFLOPS compute-bound on H800 GPUs. The introduction of FlashMLA as part of the open-source initiative reflects a commitment to transparency and community-driven innovation, allowing developers to share their progress and contribute to the collective momentum in AI exploration.'

In [22]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Can you elaborate on the features and significance of FlashMLA in the context of open-source AI development?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


FlashMLA, which stands for Fast and Scalable Machine Learning Application, is an open-source framework that facilitates rapid and efficient machine learning workflows for data scientists and engineers. It is designed to bridge the gap between complex machine learning tasks and practical, scalable implementations. Here are some key features and aspects of significance in the context of open-source AI development:

1. **Integration with Open-Source Ecosystem**: FlashMLA leverages popular Python libraries like TensorFlow, PyTorch, and Keras for building machine learning models. This integration allows developers to work within their preferred environment and seamlessly transition between different technologies based on the project requirements.

2. **Automatic Tuning and Hyperparameter Optimization**: One of the significant advantages of FlashMLA is its capability to automatically tune hyperparameters using Bayesian optimization or evolutionary algorithms. This automated tuning can signif

In [23]:
df['user_input'][123]

'Who developed the DualPipe algorithm?'

In [24]:
df['reference'][123]

'The DualPipe algorithm was created and developed by Jiashi Li, Chengqi Deng, and Wenfeng Liang.'

In [25]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Who developed the DualPipe algorithm?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The DualPipe algorithm was developed by researchers from Google AI and University of California, Berkeley. Specifically, the team involved in this development included individuals like Chris Albert, Kaimi Yao, Yujia (Jerry) Li, and Pieter Abbeel. The algorithm was designed to improve reinforcement learning in environments with delayed rewards, which is a common challenge in training agents for tasks like robot control or game playing. It was introduced in a research paper published in 2020 and has been influential in the field of reinforcement learning.<|im_end|>


In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 548 | Num Epochs = 2 | Total steps = 136
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/4,931,917,312 (0.82% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.845300
2,2.572800
3,2.687100
4,2.834200
5,2.088600
6,2.397400
7,2.041300
8,2.174000
9,2.044600
10,1.735900


In [27]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

746.0281 seconds used for training.
12.43 minutes used for training.
Peak reserved memory = 8.742 GB.
Peak reserved memory for training = 1.863 GB.
Peak reserved memory % of max memory = 59.304 %.
Peak reserved memory for training % of max memory = 12.638 %.


In [28]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "user", "content": "How does DeepSeek utilize the DualPipe method to enhance model training efficiency with H800 GPUs?"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

# outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
#                          temperature = 1.5, min_p = 0.1)
# tokenizer.batch_decode(outputs)

In [29]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Can you elaborate on the features and significance of FlashMLA in the context of open-source AI development?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

FlashMLA is a lightweight Multi-Lingual Assistant specifically designed for cold-start in weakly supervised settings, introduced as the first open-source model that surpasses DeepSeek-r1 on reasoning benchmarks, achieving a performance of 65% on AIME 2024, with impressive results without heavy RL training or large datasets. The core principle of its success lies in a sophisticated yet accessible approach, focusing on enhancing reasoning capabilities through meticulous cold-start strategies, making it an invaluable tool for developers looking to explore advanced AI functionalities while minimizing costs and resource demands.<|im_end|>


In [30]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Who developed the DualPipe algorithm?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

DualPipe was invented and created by Jiashi Li, Chengqi Deng, and Wenfeng Liang.<|im_end|>


In [31]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [32]:
import gc
gc.collect()

190

In [33]:
model.save_pretrained_gguf("lora_model_gguf", tokenizer)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 7.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.46 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 43%|████▎     | 12/28 [00:01<00:01,  9.75it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [03:37<00:00,  7.77s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving lora_model_gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving lora_model_gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving lora_model_gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving lora_model_gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at lora_model_gguf into q8_0 GGUF format.
The output location will be /content/lora_model_gguf/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: lora_model_gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.